In [ ]:
import  pandas as pd
import  numpy as np
import glob,cv2
import matplotlib.pyplot as plt
import random

import tensorflow as tf
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,BatchNormalization,Activation
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,precision_score

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing import image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# check the number of file
def check_imag_file(file_path='dataset\\normal\\*.*'):
    img_file = glob.glob(file_path)
    print(f"Total number in {file_path} is: {len(img_file)}")

# build the label for file
def build_label_filepath(data_path,label=0):
    file_list=[]
    file_iterator=glob.glob(data_path)
    for file in file_iterator:
        file_list.append([file,label])
    return file_list

# read the img file by cv
def preprocessing_image(filepath,resize_num=196):
    img = cv2.imread(filepath) #read
    img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR) #convert
    img = cv2.resize(img,(resize_num,resize_num))  # resize
    img = img / 255 #scale
    return img

# create format data
def create_format_dataset(dataframe):
    X = []
    y = []
    for f, t in dataframe.values:
        X.append(preprocessing_image(f))
        y.append(t)

    return np.array(X), np.array(y)

# set model
def build_model(X):
    model = Sequential()

    model.add(Conv2D(128, (2, 2), input_shape=(X.shape[1], X.shape[2], 3), activation='relu'))
    model.add(Conv2D(64, (2, 2), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(32, (2, 2), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(64, (2, 2), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(32, (2, 2), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(64, (2, 2), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(32, (2, 2), activation='relu'))
    model.add(MaxPooling2D())

    model.add(Flatten())
    model.add(Dense(128))
    model.add(Dense(1, activation="sigmoid"))
    #print(model.summary())
    return model


In [ ]:
dataset_path="./drive/MyDrive/Colab Notebooks/Fire-Detection/"
fire_path=dataset_path+'1/*.*'
normal_path=dataset_path+'0/*.*'
check_imag_file(fire_path)
check_imag_file(normal_path)

Total number in ./drive/MyDrive/Colab Notebooks/Fire-Detection/1/*.* is: 415
Total number in ./drive/MyDrive/Colab Notebooks/Fire-Detection/0/*.* is: 541


In [ ]:
fire_file_list=build_label_filepath(normal_path,label=0)
normal_file_list=build_label_filepath(fire_path,label=1)
total_file_list=fire_file_list+normal_file_list
random.shuffle(total_file_list)
df_total_file = pd.DataFrame(total_file_list, columns=['files', 'target'])

X, y = create_format_dataset(df_total_file)
print(X.shape,y.shape)

(956, 196, 196, 3) (956,)


In [ ]:
model=build_model(X)

train_epochs=200
callbacks = [EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)]
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# train model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=train_epochs, batch_size=32, callbacks=callbacks)

y_pred = model.predict(X_test)

Epoch 1/200
21/21 [==============================] - 11s 336ms/step - loss: 0.6858 - accuracy: 0.5785 - val_loss: 0.6598 - val_accuracy: 0.5923
Epoch 2/200
21/21 [==============================] - 5s 221ms/step - loss: 0.5611 - accuracy: 0.6846 - val_loss: 0.4068 - val_accuracy: 0.8293
Epoch 3/200
21/21 [==============================] - 5s 224ms/step - loss: 0.3413 - accuracy: 0.8640 - val_loss: 0.5210 - val_accuracy: 0.7700
Epoch 4/200
21/21 [==============================] - 5s 220ms/step - loss: 0.3712 - accuracy: 0.8326 - val_loss: 0.2438 - val_accuracy: 0.8990
Epoch 5/200
21/21 [==============================] - 5s 220ms/step - loss: 0.2317 - accuracy: 0.9028 - val_loss: 0.2957 - val_accuracy: 0.8711
Epoch 6/200
21/21 [==============================] - 5s 222ms/step - loss: 0.2127 - accuracy: 0.9118 - val_loss: 0.1846 - val_accuracy: 0.9303
Epoch 7/200
21/21 [==============================] - 5s 223ms/step - loss: 0.1861 - accuracy: 0.9268 - val_loss: 0.2081 - val_accuracy: 0.923

In [ ]:
y_pred_reshape = y_pred.reshape(-1)
fire_thresold=0.65
y_pred_reshape[y_pred_reshape<fire_thresold] = 0
y_pred_reshape[y_pred_reshape>=fire_thresold] = 1
y_pred_reshape = y_pred_reshape.astype('int')
print(classification_report(y_test, y_pred_reshape))

              precision    recall  f1-score   support

           0       0.92      0.96      0.94       162
           1       0.95      0.90      0.92       125

    accuracy                           0.93       287
   macro avg       0.94      0.93      0.93       287
weighted avg       0.93      0.93      0.93       287



In [ ]:
import pickle
pickle.dump(model, open("fire_detection_conv2d.pkl", 'wb'))